<h1 style="text-align: center;">Option Pricing</h1>

---

This project explores the use of Monte Carlo methods for pricing European and binary call options under the risk-neutral measure by simulating the underlying stock dynamics, using various numerical schemes.  
The first section provides a brief introduction to Monte Carlo techniques and their application in option pricing. It is followed by an overview of the three simulation methods used to generate stock price paths: Euler–Maruyama, Milstein, and the closed-form solution. The convergence of the first two will be discussed and numerically measured as well.
Next, the features and payoffs of European and binary call options are discussed, along with their pricing using the Black–Scholes framework.
The implementation of Monte Carlo pricing for both option types is then presented, with results systematically compared to the analytical Black–Scholes benchmarks.  
Subsequently, key model parameters are varied individually to investigate their impact on option prices. The effects are analyzed in detail, with comparisons across the different numerical schemes and the exact solution. The final section examines the asymptotic behavior of option prices and their dependence on moneyness.  
The project concludes with a summary of key findings and insights, followed by a references section.

<div style="font-size:20px; font-weight: bold;">Index</div>

1. **Introduction to Monte Carlo Methods**
2. **Simulation Methods for Underlying Stock Price**
    - 2.1 Initial parameters
    - 2.2 Implementation of Stock Price Simulation Methods
    - 2.3 Convergence
3. **European and Binary Call Options**
    - 3.1 Pricing Options
    - 3.2 Methods Validation
4. **Sensitivity Analysis**
    - 4.1 Analysis Framework
5. **Results**
    - 5.1 Volatility
    - 5.2 Risk-free rate
    - 5.3 Time to expiry
    - 5.4 Spot price
    - 5.5 Strike price
    - 5.6 Number of simulations
    - 5.7 Number of time steps
6. **Asymptotic Behaviours**
    - 6.1 Volatility
    - 6.2 Risk-free rate 
    - 6.3 Time to expiry
    - 6.4 Spot and Strike price
7. **Moneyness Analysis**
    - 7.1 Volatility
    - 7.2 Risk-free rate 
    - 7.3 Time to expiry
8. **Conclusions**
9. **References**


# 1. Introduction to Monte Carlo Methods
Monte Carlo (MC) methods are a class of computational algorithms that rely on repeated random sampling to obtain numerical results. The core idea is to simulate a large number of possible paths for the underlying asset's price and then compute the expected payoff of the option under these scenarios. This expected payoff is then discounted back to the present value using the risk-free rate, providing an estimate of the option's price.

Monte Carlo methods for option pricing are highly flexible and effective in handling complex, high-dimensional, and path-dependent contracts with minimal modeling effort. Their accuracy improves with more simulations, but they can be computationally intensive for low-dimensional problems and may converge slowly compared to analytical or PDE-based methods.

The general formula for the value of an option using MC simulation under the risk-neutral measure $\mathbb{Q}$ is:

$$
V(S, t) = e^{-r(T - t)}\, \mathbb{E}^\mathbb{Q} \,[\,\text{Payoff}\,]
$$

where:
- $V(S, t)$ is the option price at time $t$ with underlying price $S$,
- $r$ is the risk-free interest rate,
- $T$ is the time to maturity,
- $\mathbb{E}^\mathbb{Q}$ denotes the expectation under the risk-neutral measure.


Monte Carlo methods are inherently based on finite sampling, meaning the estimated price is subject to statistical noise. However, the accuracy of these methods improves as the number of simulations increases, with the error typically decreasing at a rate proportional to:

$$
\text{Error} \sim \frac{1}{\sqrt{N}}
$$

where $N$ is the number of simulated paths. This makes MC simulation a powerful, though computationally expensive, tool for pricing options — especially when analytical solutions are not available.

In summary, MC methods are highly adaptable and particularly effective for pricing complex derivatives involving many risk factors or path dependencies. They require relatively little modeling effort and can easily incorporate changing assumptions. However, they may be less efficient than analytical methods when such solutions are available and suitable.

# 2. Simulation Methods for Underlying Stock Price

The underlying stock price $S_t$ is assumed to follow the geometric Brownian motion described by the stochastic differential equation: $$dS_t = r S_t dt + \sigma S_t dW_t$$
Three methods are considered for simulating the evolution of the stock price:

1. **Closed-Form Solution**:  
   The geometric Brownian motion model admits an exact solution:

   $$S_T = S_0 \exp{ \left[(r - \frac{1}{2} \sigma^2)T + \sigma W_T \right]}$$

   where $W_T$ is a standard Brownian motion at expiry $T$.  
   This can be reformulated iteratively to simulate the entire path:
   $$S_{i+1} = S_i \cdot \exp{\left[ \left(r-\frac{1}{2}\sigma^2\right)\Delta t + \sigma \Delta W_i\right]}$$
   where $\Delta W_i = W_{i+1} - W_i$ is a Wiener process increment. 

2. **Euler-Maruyama Scheme**:  
   The Euler-Maruyama method is a simple and widely used approach for approximating solutions to stochastic differential equations. The method is computationally efficient and easy to implement, but it may require small time steps for accurate results, particularly when dealing with long time horizons or high volatility. The scheme is given by:

   $$S_{i+1} = S_t \cdot \left[1 + r \Delta t + \sigma \Delta W_i \right]$$

   Applying the recursive equation sequentially for M time steps, the stock price at expiry can be obtained as:
   $$S_T = S_0 \cdot \prod_{i=0}^{M-1} \left(1 + r\,\Delta t + \sigma\,\Delta W_i \right)$$


3. **Milstein Scheme**:  
   The Milstein method extends the Euler-Maruyama scheme by including a higher-order term, improving its accuracy. This method is particularly useful when modeling derivative prices or path-dependent options, although it is slightly more complex to implement and computationally more expensive. The Milstein scheme for the same SDE is:
   
   $$S_{i+1} = S_i \cdot \left[1 + \left(r - \frac{1}{2} \sigma^2 \right) \Delta t + \sigma \, \Delta W_i + \frac{1}{2}\sigma^2 {\left( \Delta W_i\right)}^2\right]$$

   The terminal stock price can then be expressed as:
   $$S_T = S_0 \cdot \prod_{i=0}^{M-1} \left[1 + r\,\Delta t + \sigma\,\Delta W_i + \tfrac{1}{2} \sigma^2 \left( (\Delta W_i)^2 - \Delta t \right) \right]$$

## 2.1 Initial parameters
This is the list of base parameters that will be used for the analysis:

| Parameter                |Symbol     |Value    |
|:-------------------------|:---------:|:-------:|
| Today’s stock price      | $S_0$     | 100     |
| Strike price             | $E$       | 100     |
| Time to expiry           | $T - t$   | 1 year  |
| Volatility               | $\sigma$  | 20%     |
| Risk-free interest rate  | $r$       | 5%      |

## 2.2 Implementation of Stock Price Simulation Methods

The `MonteCarlo` class enables simulation of stock price paths using three different methods: Euler-Maruyama, Milstein, and the closed-form solution.  
Each method relies on the properties of Wiener process increments, where each increment $\Delta W_i$ follows a normal distribution with mean 0 and variance $\Delta t$, that is, $\Delta W_i \sim \mathcal{N}(0,\Delta t)$. This can also be written as $\Delta W_t \sim \phi \sqrt{\Delta t}$, where $\phi \sim \mathcal{N}(0,1)$ is a standard normal random variable.  

To ensure consistency across simulations, the same Brownian increments are used for all numerical schemes. Sharing the random inputs across the three methods provides a fair and controlled basis for comparison by eliminating variability due to independent sampling. Consequently, any observed differences in option pricing can be confidently attributed to the numerical properties of the methods themselves, leading to more reliable and interpretable conclusions. 

The class implements two kinds of methods, which serve different purposes:

- *Terminal value methods* (`*_terminal`): These compute only the final value of the simulated stock price at maturity $T$, and are primarily used for *weak convergence tests*. In these tests, accuracy is evaluated based on expected values, rather than on the fidelity of individual paths. A more detailed discussion on convergence analysis follows in the next section.

- *Path methods* (`*_path`): These generate the entire trajectory of the stock price over time and are used for *strong convergence tests*, which assess how closely the simulated paths track the true solution. The path is built step-by-step using the chosen numerical scheme, and the output is a full time series of stock prices for each simulation. The initial value $S_0$ is explicitly included as the first row to represent time zero.

The vectorization procedure allows to replace explicit Python loops with array-based operations, speeding up the simulation process. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm
import statsmodels.api as sm
from IPython.display import display, HTML
import os

In [ ]:
class MonteCarlo:
    def __init__(self, spot, T, volatility, rate, time_steps, n_simulations):
        self.S = spot
        self.T = T
        self.vol = volatility
        self.r = rate
        self.nsims = n_simulations
        self.time_steps = time_steps

        self.dt = T / time_steps
        self.sqrt_dt = np.sqrt(self.dt)

    def generate_increment(self, rng: np.random.Generator) -> np.ndarray:
        return rng.standard_normal((self.time_steps, self.nsims)) * self.sqrt_dt

    # terminal value methods (used for weak convergence test)
    def Euler_Maruyama_terminal(self, dW: np.ndarray) -> np.ndarray:
        factors = 1 + self.r * self.dt + self.vol * dW
        return self.S * np.prod(factors, axis=0)

    def Milstein_terminal(self, dW: np.ndarray) -> np.ndarray:
        correction = 0.5 * self.vol**2 * (dW**2 - self.dt)
        factors = 1 + self.r * self.dt + self.vol * dW + correction
        return self.S * np.prod(factors, axis=0)

    def Closed_form_terminal(self, dW: np.ndarray) -> np.ndarray:
        sum_dW = np.sum(dW, axis=0)
        return self.S * np.exp((self.r - 0.5*self.vol**2) * self.T + self.vol * sum_dW)
    
    # path methods (used for strong convergence test)
    def Euler_Maruyama_path(self, dW: np.ndarray) -> np.ndarray:
        factors = 1 + self.r*self.dt + self.vol*dW
        cumprod = np.cumprod(factors, axis=0)
        return np.vstack((np.ones((1, self.nsims)), cumprod)) * self.S

    def Milstein_path(self, dW: np.ndarray) -> np.ndarray:
        correction = 0.5*self.vol**2*(dW**2 - self.dt)
        factors = 1 + self.r*self.dt + self.vol*dW + correction
        cumprod = np.cumprod(factors, axis=0)
        return np.vstack((np.ones((1, self.nsims)), cumprod)) * self.S

    def Closed_form_path(self, dW: np.ndarray) -> np.ndarray:
        increments = np.exp((self.r - 0.5*self.vol**2)*self.dt + self.vol*dW)
        cumprod = np.cumprod(increments, axis=0)
        return np.vstack((np.ones((1, self.nsims)), cumprod)) * self.S

Definition of the initial set of parameters:

In [ ]:
base_parameters = {
    'spot_price': 100,
    'strike_price': 100,
    'volatility': 0.2,
    'risk_free_rate': 0.05,
    'time_to_expiry': 1,
    'n_simulations': 100000,
    'time_steps': 252,
}

Using the initial set of parameters, 50 simulations are generated for each numerical scheme and plotted to illustrate typical trajectories of the underlying stock. The simulation uses 252 time steps, corresponding to the number of trading days in a year, ensuring that the temporal resolution aligns with daily market activity.
While many financial instruments, such as options, may depend only on the terminal value of the underlying asset, accurate simulation is still essential. Numerical schemes like Euler-Maruyama and Milstein discretize the stochastic differential equation that governs the asset's dynamics, introducing approximation errors. For example, the Euler-Maruyama method has a global weak error of order $\mathcal{O}(\Delta t)$, meaning that a finer discretization yields more precise results (even in cases where the payoff depends only on the final stock price).  

The seed for the random number generator is set once at the beginning to ensure reproducibility of the results. This means that every time the simulation is run with the same parameters, the generated Brownian increments (and thus the paths and terminal values) will be identical. This is especially important in convergence studies, debugging, and validation, as it ensures consistent comparison across numerical schemes under identical stochastic conditions.

In [ ]:
mc = MonteCarlo(
    spot=base_parameters['spot_price'],
    T=base_parameters['time_to_expiry'],
    volatility=base_parameters['volatility'], 
    rate=base_parameters['risk_free_rate'], 
    time_steps=base_parameters['time_steps'], 
    n_simulations=base_parameters['n_simulations']
)

rng = np.random.default_rng(200)
dW = mc.generate_increment(rng)

methods_path = {
    "Euler": mc.Euler_Maruyama_path(dW),
    "Milstein": mc.Milstein_path(dW),
    "Closed-form": mc.Closed_form_path(dW)
}

time = np.linspace(0, base_parameters['time_to_expiry'], base_parameters['time_steps'] + 1) 
fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

n_plot = 50

# Plot Euler-Maruyama paths
axes[0].plot(time, methods_path['Euler'][:, :n_plot], linewidth=1)
axes[0].set_title('Euler-Maruyama Simulation Paths', fontsize=12, pad=10)
axes[0].set_ylabel('Stock Price', fontsize=10)
axes[0].axhline(y=base_parameters['spot_price'], color='black', linestyle='--', linewidth=0.8)

# Plot Milstein paths
axes[1].plot(time, methods_path['Milstein'][:, :n_plot], linewidth=1)
axes[1].set_title('Milstein Simulation Paths', fontsize=12, pad=10)
axes[1].set_ylabel('Stock Price', fontsize=10)
axes[1].axhline(y=base_parameters['spot_price'], color='black', linestyle='--', linewidth=0.8)

# Plot Closed-form paths
axes[2].plot(time, methods_path['Closed-form'][:, :n_plot], linewidth=1)
axes[2].set_title('Closed-form Solution Paths', fontsize=12, pad=10)
axes[2].set_xlabel('Time (years)', fontsize=10)
axes[2].set_ylabel('Stock Price', fontsize=10)
axes[2].axhline(y=base_parameters['spot_price'], color='black', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.show()

## 2.3 Convergence

Convergence in ODEs measures how the numerical solution approaches the true solution as the time step $\Delta t \rightarrow 0$. Since ODEs are deterministic, error comes only from discretization. On the other hand, SDEs introduce randomness, requiring two convergence types. Given the exact solution at a generic time horizon $S_T^{\text{exact}}$ and the approximate solution $S_T^{\text{approx}}$, then the convergence can be:

- *Strong Convergence*: the numerical trajectory matches the true solution path, i.e. the the two match for each point of the path: $$\text{Weak error} = \mathbb{E}\left[ \, |S_T^{\text{exact}} - S_T^{\text{approx}}| \, \right] \leq C \cdot (\Delta t)^{\gamma} \tag{1}$$ where $\gamma$ is the strong order and $C$ is a generic constant.

- *Weak Convergence*: Measures how well the statistical properties of the numerical solution match those of the true solution: $$\text{Strong error} = \left| \, \mathbb{E}[S_T^{\text{exact}}] - \mathbb{E}[S_T^{\text{approx}}] \, \right| \leq C \cdot (\Delta t)^{\beta} \tag{2}$$ where $\beta$ is the weak order.

While the rigorous mathematical proofs of the convergence rates are beyond the scope of this work, it is possible to empirically verify these theoretical results through numerical experimentation. The `check_convergence` function provides this verification by computing both strong and weak errors across progressively refined time steps, plotting the error versus step size on a log-log scale and estimating the empirical convergence rates from the plot slopes. The code implementation is inspired by the technique used in https://hautahi.com/sde_simulation.  
To estimate the convergence rates of the Euler-Maruyama and Milstein schemes, a sequence of decreasing time steps is considered, given by $\Delta t = 2^{-k}$ for $k = 6, \ldots, 10$. For each value of $\Delta t$, $N$ Monte Carlo paths of the stochastic differential equation are simulated using both numerical schemes.  
The exact solution, which is analytically known in this case, is computed and used as a reference.

The weak and strong errors are defined following the definitions in equations $(1)$ and $(2)$.
The convergence rate is estimated by performing a linear regression in logarithmic scale. Specifically, $\log(\text{error})$ is regressed against $\log(\Delta t)$ using ordinary least squares, resulting in a fitted model of the form $$\log(\text{error}) \approx a + b \,\log(\Delta t)$$
The slope $b$ provides an estimate of the numerical method's convergence rate.

Note that the simulation results reported below use a simplified set of parameters:
- $S_0 = 1$
- $\sigma = 1$
- $r = 2$
- $T = 1$

This configuration enables meaningful convergence results with 100000 simulations. An attempt was made using the original parameters, but with the same number of simulations the results — particularly for the weak convergence rates — were not satisfactory. It is likely that at least one million simulations are required to obtain reliable estimates, but the available hardware was not capable of handling such computational demands. 

In [ ]:
def check_convergence(spot=1, volatility=1, rate=2, T=1, n_simulations=100000):
    """Weak and Strong convergence check"""
    min_power, max_power = 6, 10  
    dt_grid = np.array([2**-i for i in range(min_power, max_power+1)])
    
    weak_err_em, weak_err_mil = [], []
    strong_err_em, strong_err_mil = [], []
    
    for dt in dt_grid:
        time_steps = int(1/dt)
        mc_obj = MonteCarlo(spot, T, volatility, rate, time_steps, n_simulations)
        
        rng = np.random.default_rng(2025)  
        dW = mc_obj.generate_increment(rng)
        
        # Weak convergence
        exact = mc_obj.Closed_form_terminal(dW)
        weak_err_em.append(np.abs(np.mean(exact) - np.mean(mc_obj.Euler_Maruyama_terminal(dW))))
        weak_err_mil.append(np.abs(np.mean(exact) - np.mean(mc_obj.Milstein_terminal(dW))))
        
        # Strong convergence
        exact_path = mc_obj.Closed_form_path(dW)[-1]
        strong_err_em.append(np.mean(np.abs(exact_path - mc_obj.Euler_Maruyama_path(dW)[-1])))
        strong_err_mil.append(np.mean(np.abs(exact_path - mc_obj.Milstein_path(dW)[-1])))
    
    # Plotting 
    plt.figure(figsize=(10, 7))
    plt.loglog(dt_grid, weak_err_em, 'bo--', label="Euler (Weak)", markersize=8)
    plt.loglog(dt_grid, weak_err_mil, 'ro--', label="Milstein (Weak)", markersize=8)
    plt.loglog(dt_grid, strong_err_em, 'bs-', label="Euler (Strong)", markersize=8)
    plt.loglog(dt_grid, strong_err_mil, 'rs-', label="Milstein (Strong)", markersize=8)
    plt.xlabel('Time Step $\Delta t$', fontsize=12)
    plt.ylabel('Error', fontsize=12)
    plt.title('Convergence Analysis', fontsize=14)
    plt.legend(fontsize=10, loc='upper left', framealpha=1)
    plt.grid(True, which="both", ls="--", alpha=0.5)
    plt.xticks([10**-4, 10**-3, 10**-2], ['10⁻⁴', '10⁻³', '10⁻²'])
    plt.yticks([10**-2, 10**-1], ['10⁻²', '10⁻¹'])
    plt.xlim(0.0009, 0.02)
    plt.tight_layout()
    plt.show()

    # Regression analysis for all cases
    X = sm.add_constant(np.log(dt_grid))
    
    def print_convergence_rate(y, name, expected):
        results = sm.OLS(np.log(y), X).fit()
        print(f'{name}: {results.params[1]:.3f} (Expected: {expected})')
    
    print("=== Convergence Rates ===")
    print_convergence_rate(weak_err_em, "Weak Euler-Maruyama", "1.0")
    print_convergence_rate(weak_err_mil, "Weak Milstein      ", "1.0")
    print_convergence_rate(strong_err_em, "Strong Euler-Maruyama", "0.5")
    print_convergence_rate(strong_err_mil, "Strong Milstein      ", "1.0")

check_convergence()

The results obtained are reported in the following tables:
<div style="display: flex; margin-left: 500px; gap: 100px;">
  <div style="text-align: center;">
    <strong>Theoretical Convergence Rates</strong>
    <table style="border-collapse: collapse; text-align: center;">
      <thead><tr><th>Method</th><th>Strong (&gamma;)</th><th>Weak (&beta;)</th></tr></thead>
      <tbody>
        <tr><td>Euler-Maruyama</td><td>0.5</td><td>1.0</td></tr>
        <tr><td>Milstein</td><td>1.0</td><td>1.0</td></tr>
      </tbody>
    </table>
  </div>
  <div style="text-align: center;">
    <strong>Estimated Convergence Rates</strong>
    <table style="border-collapse: collapse; text-align: center;">
      <thead><tr><th>Method</th><th>Strong (&gamma;)</th><th>Weak (&beta;)</th></tr></thead>
      <tbody>
        <tr><td>Euler-Maruyama</td><td>0.515</td><td>0.997</td></tr>
        <tr><td>Milstein</td><td>0.982</td><td>0.980</td></tr>
      </tbody>
    </table>
  </div>
</div>

Looking at the tables it is possible to conclude that the estimated convergence rates align closely with the theoretical expectations.

---

# 3. European and binary call options
The payoff function depends on the type of contract:

• A **European call** option gives the holder the right, but not the obligation, to buy the underlying asset at a predetermined strike price $E$ on the expiration date $T$.  
The payoff of a European call option is:

$$\text{Payoff} = \max(S_T - E, 0)$$

where $S_T$ is the price of the underlying asset at maturity.
The Black-Scholes pricing formula for European call options is:
$$ C(S,t) = S_0 N(d_1) - E e^{-rT}N(d_2) $$
where 

$$d_1 = \frac{\ln{\left(\frac{S_0}{E}\right)} + \left(r + \frac{1}{2}\sigma^2 \right)T}{\sigma \sqrt{T}}$$

$$d_2 = d_1 - \sigma \sqrt{T}$$

$$N(x) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} e^{-\frac{t^2}{2}} \, dt$$


• A **binary call** option pays a fixed amount if the underlying asset's price is above the strike price at expiration, and nothing otherwise.  
The payoff of a binary call option is:

$$\text{Payoff} = 
\begin{cases} 
1 & \text{if } S_T \geq E \\
0 & \text{otherwise}
\end{cases}
$$
The Black-Scholes pricing formula for binary call options is:
$$ B(S,t) = e^{-rT}N(d_2)$$

## 3.1 Pricing options
The `OptionPricing` class implements pricing for European and binary options using two complementary approaches. The Monte Carlo simulation method computes option prices numerically using the previously generated asset prices, providing both value estimates and associated standard errors calculated as standard deviation of the payoff divided by the square root of the number of simulations, then discounted to present value. The standard error decreases systematically with increasing simulation count, ensuring convergence to the true value under the model’s assumptions.

For validation, the class incorporates the analytical Black-Scholes formulas, which serve as theoretical benchmarks, since the MC methods are expected to approach the BS option value as the number of simulations grows. 

In [ ]:
class OptionPricing:
    def __init__(self, terminal_prices, strike, spot, T, vol, rate):
        self.S_T = np.asarray(terminal_prices)
        self.E = strike
        self.S0 = spot
        self.T = T
        self.sigma = vol
        self.r = rate
        self.discount_factor = np.exp(-self.r * self.T)

        self.d1 = (np.log(self.S0 / self.E) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        self.d2 = self.d1 - self.sigma * np.sqrt(self.T)

    def european_mc(self):
        """ Monte Carlo estimate of European call price """
        payoff = np.maximum(self.S_T - self.E, 0)
        mean_payoff = payoff.mean()
        stderr = payoff.std(ddof=1) / np.sqrt(payoff.size)
        return {
            'price': self.discount_factor * mean_payoff,
            'stderr': self.discount_factor * stderr
        }

    def binary_mc(self):
        """ Monte Carlo estimate of binary call price """
        payoff = (self.S_T >= self.E).astype(float)
        mean_payoff = payoff.mean()
        stderr = payoff.std(ddof=1) / np.sqrt(payoff.size)
        return {
            'price': self.discount_factor * mean_payoff,
            'stderr': self.discount_factor * stderr
        }

    def bs_european(self):
        """ Black–Scholes formula for a European call """
        return self.S0 * norm.cdf(self.d1) - self.E * self.discount_factor * norm.cdf(self.d2)

    def bs_binary(self):
        """ Black–Scholes formula for binary call """
        return self.discount_factor * norm.cdf(self.d2)

In [ ]:
results = []

methods = {
    "Euler": mc.Euler_Maruyama_terminal(dW),
    "Milstein": mc.Milstein_terminal(dW),
    "Closed-form": mc.Closed_form_terminal(dW)
}

# Compute prices for each method
for method_name, term_vals in methods.items():
    opt = OptionPricing(
        terminal_prices=term_vals,
        strike=base_parameters['strike_price'],
        spot=base_parameters['spot_price'],
        T=base_parameters['time_to_expiry'],
        vol=base_parameters['volatility'],
        rate=base_parameters['risk_free_rate']
    )
    
    european = opt.european_mc()
    binary = opt.binary_mc()
    bs_european = opt.bs_european()
    bs_binary = opt.bs_binary()
    
    results.append({
        'Method': method_name,
        'European_price': european['price'],
        'European_err': european['stderr'],
        'Binary_price': binary['price'],
        'Binary_err': binary['stderr'],
        'BS European': bs_european,
        'BS binary': bs_binary,
    })

results_df = pd.DataFrame(results)
print(results_df)

The price obtained are summed up in the following table:

|Method|European|Binary|
|:---:|:---:|:---:|
|Euler-Maruyama |10.4411 $\pm$ 0.0464 | 0.5337 $\pm$ 0.0015|
|Milstein       |10.4401 $\pm$ 0.0464 | 0.5335 $\pm$ 0.0015|
|Closed form    |10.4419 $\pm$ 0.0464 | 0.5335 $\pm$ 0.0015|
|Black-Scholes  |10.4506              | 0.5323             |

All option values obtained using the three Monte Carlo methods are very close to one another, and the Black–Scholes benchmark lies within the range of variability introduced by random sampling. A more rigorous assessment of the consistency between methods is presented in the next section.

## 3.2 Methods validation

To validate the accuracy of the Monte Carlo simulations, the estimated option prices are compared with the theoretical Black-Scholes (BS) values using two metrics: the *relative difference* and the *$t$-statistic*. These measures offer complementary insights: the relative difference quantifies the numerical deviation from the benchmark, while the $t$-statistic assesses statistical compatibility.

The relative difference is defined as:

$$\text{Relative difference} = \frac{\text{Price}_{MC} - \text{Price}_{BS}}{\text{Price}_{BS}}$$

which provides a normalized measure of bias in the Monte Carlo estimate.

The $t$-statistic is given by:

$$t = \frac{|\text{Price}_{MC} - \text{Price}_{BS}|}{\text{Standard Error}_{MC}}$$

and is used to test the null hypothesis that the Monte Carlo estimate is statistically consistent with the Black-Scholes value.
- If $t > 1.96$, the difference is statistically significant at the 95\% confidence level, suggesting potential issues, like implementation errors or violated assumptions.  
- If $t \leq 1.96$, the Monte Carlo and Black-Scholes prices are statistically compatible within expected sampling variation.



In [ ]:
def perform_validation(row):
    # t-stat
    european_t = abs(row['European_price'] - row['BS European']) / row['European_err']
    binary_t = abs(row['Binary_price'] - row['BS binary']) / row['Binary_err']
    
    european_sig = european_t > 1.96
    binary_sig = binary_t > 1.96
    
    # relative difference
    european_rel_diff = abs(row['European_price'] - row['BS European']) / row['BS European']
    binary_rel_diff = row['Binary_price'] - row['BS binary'] / row['BS binary']
    
    return pd.Series({
        'European_t': european_t,
        'European_Significant': european_sig,
        'Binary_t': binary_t,
        'Binary_Significant': binary_sig,
        'European_Relative_Difference': european_rel_diff,
        'Binary_Relative_Difference': binary_rel_diff,
    })

t_test_df = results_df.apply(perform_validation, axis=1)

final_df = pd.concat([results_df, t_test_df], axis=1)

print(final_df[['Method', 'European_t', 'European_Significant', 'Binary_t', 'Binary_Significant', 
                'European_Relative_Difference', 'Binary_Relative_Difference']])

The results are summed up in the following table:

<div style="display: flex; margin-left: 500px; gap: 100px;">
  <div style="text-align: center;">
    <strong>European option</strong>
    <table style="border-collapse: collapse; text-align: center;">
      <thead><tr><th>Method</th><th>(MC - BS)/BS</th><th>t-stat</th></tr></thead>
      <tbody>
        <tr><td>Euler-Maruyama</td><td>0.0009</td><td>0.2049</td></tr>
        <tr><td>Milstein</td><td>0.0010</td><td>0.2259</td></tr>
        <tr><td>Closed-form</td><td>0.0008</td><td>0.1870</td></tr>
      </tbody>
    </table>
  </div>
  <div style="text-align: center;">
    <strong>Binary option</strong>
    <table style="border-collapse: collapse; text-align: center;">
      <thead><tr><th>Method</th><th>(MC - BS)/BS</th><th>t-stat</th></tr></thead>
      <tbody>
        <tr><td>Euler-Maruyama</td><td>0.0026</td><td>0.9254</td></tr>
        <tr><td>Milstein</td><td>0.0022</td><td>0.7852</td></tr>
        <tr><td>Closed-form</td><td>0.0022</td><td>0.7725</td></tr>
      </tbody>
    </table>
  </div>
</div>

Binary options exhibit slightly larger deviations from the benchmark Black–Scholes values, with relative differences still remaining below $0.3%$.  
Nonetheless, all $t$-statistics fall well within the 95% confidence interval, reinforcing the evidence of accurate and reliable model performance.

---

# 4. Sensitivity analysis
To assess the impact of each model parameter on option pricing, a sensitivity analysis is conducted by varying one parameter at a time while keeping the others fixed. This approach isolates the individual contribution of each input to the final price. The three numerical methods are evaluated against the Black–Scholes benchmark, considering both analytical values and the results of a statistical $t$-test, as well as the relative difference between the MC and BS prices.

Although the number of simulations and time steps are not intrinsic model parameters, their influence on the results is also discussed at the end of this section.

## 4.1 Analysis framework

The following functions implement the complete sensitivity analysis workflow. Each function is designed to perform a specific task in the pipeline — from running simulations and generating plots to creating comparison tables and displaying results. The modular structure improves readability, facilitates debugging, and allows individual components to be tested or reused independently.  

The baseline parameter set is the same as that reported at the beginning of the report, with a fixed number of 100000 simulations and a time step of $\Delta t=1/252$.

Two configuration dictionaries, `PARAM_KEY_MAP` and `PARAM_DISPLAY`, establish parameter naming conventions and display formatting.

In [ ]:
# Map parameter 
PARAM_KEY_MAP = {
    'volatility': 'volatility',
    'rate': 'risk_free_rate',
    'expiry': 'time_to_expiry',
    'spot': 'spot_price',
    'strike': 'strike_price',
    'nsim': 'n_simulations',
    'nsteps': 'time_steps'
}

# Map parameters for plotting
PARAM_DISPLAY = {
    'volatility': (r'Volatility ($\sigma$)', 'Volatility'),
    'rate': ('Risk-Free Rate (r)', 'Risk-Free Rate'),
    'expiry': ('Time to Expiry (T) [years]', 'Time to Expiry'),
    'spot': (r'Spot Price ($S_0$)', 'Spot Price'),
    'strike': ('Strike Price (E)', 'Strike Price'),
    'nsim': ('Number of Simulations', 'Simulation Count'),
    'nsteps': ('Number of Time Steps', 'Time Steps')
}

The `run_simulations` function performs the sensitivity analysis by taking as input a set of base parameters, the name of the parameter to be varied, and a list of values that this parameter will assume during the analysis. It systematically executes the full workflow for sensitivity testing.

The function first initializes containers to store results for the Euler-Maruyama, Milstein, and closed-form methods. For each value in the parameter sweep, the base configuration is updated accordingly. Asset price terminal values are then simulated using all three methods. For each method and parameter value, the function computes both European and binary option prices, along with their corresponding standard errors. These MC results are stored together with the analytical BS benchmark values for subsequent comparison.

The function returns a dictionary containing the obtained results.

In [ ]:
def run_simulations(params_base, param_name, param_values):
    methods = ['Euler', 'Milstein', 'Closed_form']
    results = {m: {'values': [], 'european_price': [], 'european_stderr': [], 'european_BS': [],
                   'binary_price': [], 'binary_stderr': [], 'binary_BS': []}
               for m in methods}
    rng = np.random.default_rng(params_base.get('seed', None))
    for val in param_values:
        params = params_base.copy()
        key = PARAM_KEY_MAP[param_name]
        params[key] = float(val)
        mc = MonteCarlo(
            spot=params['spot_price'], 
            T=params['time_to_expiry'],
            volatility=params['volatility'], 
            rate=params['risk_free_rate'],
            time_steps=int(params['time_steps']), 
            n_simulations=int(params['n_simulations'])
        )
        
        dW = mc.generate_increment(rng)
        terminals = [
            mc.Euler_Maruyama_terminal(dW),
            mc.Milstein_terminal(dW),
            mc.Closed_form_terminal(dW)
        ]

        for method, term in zip(methods, terminals):
            opt = OptionPricing(
                terminal_prices=term,
                strike=params['strike_price'], 
                spot=params['spot_price'],
                T=params['time_to_expiry'], 
                vol=params['volatility'], 
                rate=params['risk_free_rate']
            )
            results[method]['values'].append(val)

            mc_eu = opt.european_mc(); bs_eu = opt.bs_european()
            mc_bin = opt.binary_mc(); bs_bin = opt.bs_binary()

            results[method]['european_price'].append(mc_eu['price'])
            results[method]['european_stderr'].append(mc_eu['stderr'])
            results[method]['european_BS'].append(bs_eu)
            results[method]['binary_price'].append(mc_bin['price'])
            results[method]['binary_stderr'].append(mc_bin['stderr'])
            results[method]['binary_BS'].append(bs_bin)

    return results

The `plot_option_results` function generates plots that visualize the sensitivity of option prices to variations in a specific model parameter. It takes as input the results dictionary from the `run_simulations` function, the name of the parameter being analyzed, and the type of option under consideration.

The function constructs a plot where the x-axis represents the values of the selected parameter and the y-axis corresponds to the computed option prices. For each method, the MC prices are displayed with error bars to reflect confidence intervals ($\text{Std\_err} \cdot 1.96$). Additionally, the analytical BS reference price is shown using red cross markers for comparison.

In [ ]:
def plot_option_results(results, param_name, opt_type, base_dir):
    methods = list(results.keys())
    plt.figure(figsize=(8, 6), dpi=300)

    for method in methods:
        vals = results[method]['values']
        plt.errorbar(
            vals,
            results[method][f'{opt_type}_price'],
            yerr=1.96 * np.array(results[method][f'{opt_type}_stderr']),
            linestyle='-', marker='None', capsize=4,
            label=f'{method} MC'
        )

    bs_vals = results[methods[0]]['values']
    plt.plot(
        bs_vals,
        results[methods[0]][f'{opt_type}_BS'],
        linestyle='', 
        marker='x', 
        markersize=6,
        zorder=3, 
        label='BS Reference'
    )

    # plotting
    plt.title(f'{opt_type.capitalize()} Call Option')
    plt.xlabel(PARAM_DISPLAY[param_name][0])
    plt.ylabel('Option Price')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    
    # save plots
    out_dir = os.path.join(base_dir, f"{param_name}")
    os.makedirs(out_dir, exist_ok=True)

    filename = f"{param_name}_{opt_type}.png"
    filepath = os.path.join(out_dir, filename)
    plt.savefig(filepath, bbox_inches='tight')
    plt.close()

    return filepath

The `generate_option_table` function creates a comprehensive summary table to evaluate the accuracy of Monte Carlo methods against the Black-Scholes benchmark. It computes their relative price difference, as well as the corresponding $t$-statistic.

For each combination of method and parameter value, the table records the parameter setting, the method used, the MC price and its standard error, the BS price, their relative difference, and the associated $t$-statistic. 

The table employs a color-coding scheme to enhance interpretability: the column showing the relative difference is shaded with a symmetric red gradient, where deeper hues indicate larger deviations from the Black-Scholes price. In contrast, the $t$-statistic column is visualized using a blue gradient to reflect the statistical significance of the price difference.

In [ ]:
def generate_option_table(results, param_name, opt_type):
    methods = list(results.keys())
    rows = []
    for m in methods:
        for i, val in enumerate(results[m]['values']):
            mc_p = results[m][f'{opt_type}_price'][i]
            mc_e = results[m][f'{opt_type}_stderr'][i]
            bs_p = results[m][f'{opt_type}_BS'][i]
            rel = (mc_p - bs_p) / bs_p if bs_p else float('nan')
            t   = abs(mc_p - bs_p) / mc_e if mc_e else float('nan')
            rows.append({
                'Param': val,
                'Method': m,
                'MC Price': mc_p,
                '± StdErr': mc_e,
                'BS Price': bs_p,
                '(MC - BS)/BS': rel,
                't-stat': t
            })

    df = pd.DataFrame(rows).rename(columns={'Param': PARAM_DISPLAY[param_name][1]})
    for col in ['MC Price', '± StdErr', 'BS Price', '(MC - BS)/BS', 't-stat']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    absmax = df['(MC - BS)/BS'].abs().max()
    def color_rel(v):
        norm = abs(v) / absmax if absmax else 0
        rgba = matplotlib.colormaps['YlOrRd'](0.5 + norm/2)
        return f'background-color: {matplotlib.colors.rgb2hex(rgba)}'
    
    def add_method_separator(row):
        # Put a thick border on top of the first row of each method
        if row.name == 0 or df.iloc[row.name]['Method'] != df.iloc[row.name - 1]['Method']:
            return ['border-top: 3px solid black' for _ in row]
        else:
            return ['' for _ in row]
    
    styled = df.style.set_caption(
            f"<h3 style='text-align:center'>{opt_type.capitalize()} Option – {PARAM_DISPLAY[param_name][1]} Sensitivity</h3>") \
        .map(color_rel, subset=['(MC - BS)/BS']) \
        .background_gradient(subset=['t-stat'], cmap='Blues') \
        .format({
            PARAM_DISPLAY[param_name][1]: '{:.3f}',
            'MC Price': '{:.4f}',
            '± StdErr': '{:.4f}',
            'BS Price': '{:.4f}',
            '(MC - BS)/BS': '{:.4f}',
            't-stat': '{:.2f}'
        }, na_rep='N/A') \
        .hide(axis='index') \
        .apply(add_method_separator, axis=1) \
        .set_properties(**{'text-align': 'center'})
    html = styled.to_html()

    return styled, html

`display_plots` and `display_tables` just handle the layout of plots and tables respectively.

In [ ]:
import time

def display_plots(param_name, base_dir):
    folder = os.path.join(base_dir, f"{param_name}")
    euro_img = os.path.join(folder, f"{param_name}_european.png")
    bin_img  = os.path.join(folder, f"{param_name}_binary.png")

    cache_buster = f"?v={int(time.time())}"

    html = f"""
    <div style='display:flex;justify-content:center;gap:40px;margin-top:20px;'>
      <div style='flex:1;text-align:center;'>
        <img src='{euro_img}{cache_buster}' style='width:100%;height:auto;' />
      </div>
      <div style='flex:1;text-align:center;'>
        <img src='{bin_img}{cache_buster}' style='width:100%;height:auto;' />
      </div>
    </div>
    """
    display(HTML(html))

In [ ]:
def display_tables(tables):
    html = f"""
    <div style="position: relative; left: 210px; width: calc(100% - 250px);">
    
      <div style="
          display: flex;
          justify-content: flex-start;  
          align-items: flex-start;
          gap: 260px;
          margin-top: 20px;
          width: 100%;
        ">
        
        <style>
          .dataframe {{
            margin: 0 !important;
            border-collapse: collapse;
          }}
        </style>
        
        <div style="width: calc(50% - 20px); min-width: 300px;">
          {tables['european']}
        </div>
        <div style="width: calc(50% - 20px); min-width: 300px;">
          {tables['binary']}
        </div>
      </div>
    </div>
    """
    display(HTML(html))


The `parameter_analysis` function coordinates the entire sensitivity analysis workflow for a selected model parameter, using all the functions just mentioned.

In [ ]:
def parameter_analysis(params_base, param_name, param_values, base_dir):
    
    results = run_simulations(params_base, param_name, param_values)

    tables = {}
    for t in ['european', 'binary']:
        plot_option_results(results, param_name, t, base_dir)
        _, html = generate_option_table(results, param_name, t)
        tables[t] = html

    display_plots(param_name, base_dir)
    display_tables(tables)

    return results, tables

---

# 5. Results

The results of the sensitivity analysis are presented and discussed below. Note that the error bars in the plots may appear either too small or too large due to the scale of the y-axis. To address this, accompanying tables are also provided, which report both the relative distance between the Monte Carlo estimates and the Black–Scholes values, as well as the outcome of the corresponding $t$-tests.

## 5.1 Volatility 

In [ ]:
MAIN_PLOTS = 'sensitivity_analysis'

vol_results, vol_tables = parameter_analysis(
    params_base=base_parameters,
    param_name='volatility',
    param_values=np.linspace(0.1, 1.0, 10),
    base_dir=MAIN_PLOTS
)

The volatility range used in this analysis is between $10\%$ and $100\%$.

**European option**: The value increases with volatility almost linearly within the chosen parameter space. 
Simulated values obtained via MC methods are statistically compatible with the BS price at the $95\%$ confidence level, and the relative difference between MC and BS values is typically below $1\%$.
All three methods provide almost identical results, without anyone outperforming the others.

**Binary option**: Exhibits a decreasing price as volatility increases. As a matter of fact, the distribution spreads out, and although some paths end higher than $E$, more also fall below $E$, so the net probability of finishing in-the-money drops. The three numerical methods yield comparable results, with larger discrepancies appearing at high volatility levels—though these differences remain within the range of statistical fluctuations.  

The standard error provides additional insight into the behavior of the Monte Carlo estimates. The key difference in how binary and European call options respond to changes in volatility stems from the structure of their payoffs. A binary call option has a bounded payoff, taking values only in ${0, 1}$, which inherently limits its variance regardless of the volatility level. Consequently, the standard error in Monte Carlo simulations remains well-controlled even as $\sigma$ increases, with a theoretical upper bound of $\frac{e^{-rT}}{2\sqrt{N}} \approx 0.0015$.

## 5.2 Risk-free rate 

In [ ]:
rate_results, rate_tables = parameter_analysis(
    params_base=base_parameters,
    param_name='rate',
    param_values=np.linspace(0.04, 0.06, 10),
    base_dir=MAIN_PLOTS
)

The risk-free rate values range for the analysis is $4-6\%$ 

**European option**: The option value grows with increasing risk-free rate $r$. It is observed a rather stable value of the standard error across the different $r$ values, and the discrepancy between MC and BS are below $0.4\%$. The $t$-test confirms that all three methods work well.

**Binary option**: The option value grows as well, following a growing monotone slightly concave curve. The relative difference between prices is low and the $t$-stat values confirm a good agreement across all methods, with local peaks probably stemming from statistical noise. 

## 5.3 Time to expiry

In [ ]:
expiry_results, expity_tables = parameter_analysis(
    params_base=base_parameters,
    param_name='expiry',
    param_values=np.linspace(0.02, 2, 10),
    base_dir=MAIN_PLOTS
)

The chosen interval covers expirations from $1$ week to $2$ years.

**European option**: The option value grows for longer periods to expiry monotonically. This because the more the expiry is distant in the future, the more time option has to get in-the-money. Even though the $t$-stat values are in average a bit higher than the ones seen so far, they are still acceptable at 95% confidence level. Also relative differences don't ever go above $0.6\%$.

**Binary option**: There is a rapid increase in value up to $T \approx 1$ and then it falls back. The time to maturity at which a binary call option reaches its maximum price depends on a combination of the option’s moneyness, the risk-free interest rate, and the volatility of the underlying asset. For at-the-money options, the price initially increases with time as the probability of expiring in the money, captured by the cumulative distribution $N(d_2)$, grows. However, as maturity extends further, the exponential discount factor $e^{−rT}$ begins to dominate and pulls the price downward. The precise location of the peak reflects this tradeoff: a higher risk-free rate accelerates discounting and shifts the maximum earlier, while higher volatility flattens the distribution and can push the maximum later or make it less distinct. 
Deep in-the-money or out-of-the-money options tend not to exhibit a pronounced peak, since either the probability or the discounting clearly dominates across maturities.  
All three methods give good and comparable results, with restrained price difference between MC and BS and $t$-stat values well within the acceptance threshold, despite an isolated spike at $T=2$.

## 5.4 Spot price

In [ ]:
spot_results, spot_tables = parameter_analysis(
    params_base=base_parameters,
    param_name='spot',
    param_values=np.linspace(60, 140, 9),
    base_dir=MAIN_PLOTS
)

The range goes from $S=60$ to $S=140$.

**European option**: As the initial spot price $S_0$ increases, the option value grows with increasing pace. This occurs because a higher $S_0$ increases the probability that the option will expire in-the-money (i.e. $S_T > K$).  
For deep out-the-money options, there is a noticeable discrepancy between the MC and BS results due to the small option values (for both European and binary options). In this case, both the numerator and denominator are close to zero, which leads to higher relative errors, even though the $t$-test does not indicate a significant difference. In general, all other values are all within acceptable statistical bounds. 

**Binary option**: The behavior of a binary option is similar, but it flattens out more deeply when in-the-money. This occurs because, when $S_0$ is significantly above $K$, the probability of expiring in-the-money approaches $1$, and further increases in $S_0$ offer minimal additional value. At expiry, this curve is just a Heaviside step function $\mathcal{H}(S_T-K)$.
The same observations made about relative difference and t-stat work here as well.  

In terms of performance, the three methods yield very similar results for both options. 

## 5.5 Strike price

In [ ]:
strike_results, strike_tables = parameter_analysis(
    params_base=base_parameters,
    param_name='strike',
    param_values=np.linspace(60, 140, 9),
    base_dir=MAIN_PLOTS
)

This is essentially a mirror image of the previous analysis. Whereas before the spot price $S_0$ varied while keeping the strike price $E$ fixed, now $S_0$ is fixed and $E$ varies. Despite this change in perspective, the relationship between the option value and moneyness remains the same. As $E$ increases relative to the fixed $S_0$, the option becomes less likely to finish in the money, reducing its value — just as decreasing $S_0$ did in the earlier case. 
A larger relative discrepancy between MC and BS values is again observed for deep out-the-money options.

## 5.6 Number of simulations

In [ ]:
sim_results, sim_tables = parameter_analysis(
    params_base=base_parameters,
    param_name='nsim',
    param_values=np.linspace(1000, 100000, 10).astype(int),
    base_dir=MAIN_PLOTS
)

The number of simulations performed span from 1000 to 100000.

**European option**: In this case the option price is supposed to remain the same, but the accuracy increases. This can be seen by the standard error, which scales as $1/\sqrt{N}$. The relative distance between MC and BS values is relevant only for 1000 simulations, and then becomes stable. The standard error starting at $\sim 45\%$ drops to $\sim 4.7\%$ for 100000 events. 

**Binary option**: The same arguments work here as well, with the capped standard error decreasing at rate $1 / \sqrt{N}$, and reaching the already seen value of $0.15\%$ for 100000 simulations.

## 5.7 Number of time steps

In [ ]:
steps_results, steps_tables = parameter_analysis(
    params_base=base_parameters,
    param_name='nsteps',
    param_values=np.linspace(1, 600, 10).astype(int),
    base_dir=MAIN_PLOTS
)

Both European and binary options are path-independent, as they only depend on the value at expiry. Despite this, a convergence trend is observed as the number of time steps increases.

**European option**: Euler and Milstein methods fail to provide acceptable results at the 95% confidence level for one time step, but rapidly improve with more steps, while the closed-form solution remains stable. This behavior likely stems from the limited accuracy of the discretization, which introduces a bias of order $\mathcal{O}(\Delta t)$. In the case of a single step, this results in a non-negligible discrepancy from the Black–Scholes benchmark.

**Binary option**: In this case the Milstein method seems to perform as well as the closed form solution, while the Euler method still fails to provide an acceptable result.

Here is a possible explanation: Milstein captures the mean and variance of $S_T$ correctly to leading order in $\Delta t$, yielding a good estimate of the crossing probability. On the other hand, Euler’s incorrect variance leads to a biased estimate of $\mathbb{P}(S_T \ge K)$. For the European call, however, the expected payoff is sensitive not only to mean and variance but also to higher moments of the distribution. While Milstein has weak order 1, the error from a single large step introduces a bias of order $\mathcal{O}(1)$, which is significant for this type of payoff. As a result, more time steps are required to accurately capture the shape of the payoff distribution and reduce this bias.

---

# 6. Asymptotic behaviours 

In this section the asymptotic behaviour of the option prices is explored. Here the scope is just to show the theoretical curve obtained for the option prices under the Black-Scholes framework, and the parameters reach values far beyond the real world scenarios. Therefore, the MC results are not reported, and only plots will be retained, since tables won't be necessary.

In [ ]:
ASYMPTOTIC_DIR = 'asymptotic_plots'

def asymptotic_analysis(params_base, param_name, param_values, base_dir=ASYMPTOTIC_DIR):
    
    if param_name not in PARAM_KEY_MAP:
        raise ValueError(f"Unsupported parameter: {param_name}")

    out_dir = os.path.join(base_dir, f"{param_name}")
    os.makedirs(out_dir, exist_ok=True)

    euro_vals = []
    binary_vals = []

    for val in param_values:
        p = params_base.copy()
        p[PARAM_KEY_MAP[param_name]] = float(val)

        opt = OptionPricing(
            terminal_prices=None,
            strike=p['strike_price'],
            spot=p['spot_price'],
            T=p['time_to_expiry'],
            vol=p['volatility'],
            rate=p['risk_free_rate']
        )
        euro_vals.append(opt.bs_european())
        binary_vals.append(opt.bs_binary())

    # European plot
    plt.figure(figsize=(8, 6), dpi=300)
    plt.plot(param_values, euro_vals, marker='', linestyle='-')
    plt.title(f"European Call – {PARAM_DISPLAY[param_name][1]} Sensitivity")
    plt.xlabel(PARAM_DISPLAY[param_name][0])
    plt.ylabel('Option Price')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    euro_path = os.path.join(out_dir, f"{param_name}_european.png")
    plt.savefig(euro_path, bbox_inches='tight')
    plt.close()

    # Binary plot
    plt.figure(figsize=(8, 6), dpi=300)
    plt.plot(param_values, binary_vals, marker='', linestyle='-')
    plt.title(f"Binary Call – {PARAM_DISPLAY[param_name][1]} Sensitivity")
    plt.xlabel(PARAM_DISPLAY[param_name][0])
    plt.ylabel('Option Price')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    binary_path = os.path.join(out_dir, f"{param_name}_binary.png")
    plt.savefig(binary_path, bbox_inches='tight')
    plt.close()

    return {'european': euro_path, 'binary': binary_path}

## 6.1 Volatility

In [ ]:
asymptotic_analysis(
    params_base=base_parameters,
    param_name='volatility',
    param_values=np.linspace(0.1, 8.0, 30),
)

display_plots('volatility', ASYMPTOTIC_DIR)

**European option**: The value increases with volatility and approaches a plateau for $\sigma \gtrsim 5$. According to the Black-Scholes pricing formula, this behavior is expected: as $\sigma \to \infty$, the call price tends to $S_0$, since $N(d_1) \to 1$ and $N(d_2) \to 0$.

**Binary option**: The value is essentially proportional to $N(d_2)$, with the discount factor acting only as a constant scaling term when $r$ and $T$ are fixed, and therefore tends to zero as $\sigma$ increases.

## 6.2 Risk-free rate

In [ ]:
asymptotic_analysis(
    params_base=base_parameters,
    param_name='rate',
    param_values=np.linspace(0.01, 10.0, 30),
)

display_plots('rate', ASYMPTOTIC_DIR)

**European option**: The value asymptotycally approaches $S_0$. This can be seen from the Black-Scholes pricing fomrula, where the discounted term vanishes for  
$r \to \infty$ and $N(d_1) \to 1$.

**Binary option**: At first the $N(d_2)$ dominates, leading to a growth in value, but as $r \to \infty$ the discounting factor wins and the option value falls towards zero.

## 6.3 Time to expiry

In [ ]:
asymptotic_analysis(
    params_base=base_parameters,
    param_name='expiry',
    param_values=np.linspace(0.2, 30.0, 30),
)

display_plots('expiry', ASYMPTOTIC_DIR)

**European option**: The behavior of the option price as time to expiry varies mirrors the considerations made in the limit of the risk-free rate, resulting in a plateau at  
$S_0$. However, the convergence is slower in this case due to the asymptotic behavior $$d_1(T) \sim \frac{T}{\sqrt{T}}$$ in contrast to the linear dependence $d_1(T) \sim r$ observed in the risk-free rate analysis.

**Binary option**: For similar reasons, the option price initially increases and then decreases as expiry grows, but at a different rate due to the distinct asymptotic behavior of the relevant terms.

## 6.4 Spot and Strike price

Using a wider range than the the one used before $(60 \leq S_0 \leq 140)$ wouldn't add anything interesting. In turn, it is worth looking at the shape of the option value versus spot price as the time to expiry gets smaller. 

In [ ]:
base_parameters['time_to_expiry'] = 0.001

spot_lim2, spot_tables_lim2 = asymptotic_analysis(
    params_base=base_parameters,
    param_name='spot',
    param_values=np.linspace(60, 140, 100),
)

display_plots('spot', ASYMPTOTIC_DIR)

In both cases the option value is very close to the payoff function, with a time to expiry of less than 9 hours.

**European option**: With a very close the expiration date, the value curve tends to a limit where the option price is zero for $S_T \leq K$ and enters a liner regime for $S_T > K$, corresponding to the option's payoff structure.

**Binary option**: The option price curve approaches a step function, where the step happens at the strike price. A finer granularity was used to improve the quality of the plot (more points help to visualize the actual behaviour of the price function, which is discontinuous).

The same thing can be seen when studying the variation of the strike price, but the plots will be the mirror image with respect to the vertical axis placed at $E=100$.

---

# 7. Moneyness analysis

Up to this point, the analysis has assumed the initial condition $S_0 = E = 100$ for the underlying stock. It is insightful to investigate how option prices behave when this initial condition is varied, while repeating the parameter sweeps over volatility, risk-free rate, and time to maturity.

To explore this, 3D surface plots are generated with axes representing moneyness (defined as $S_0 / E$), the selected Black-Scholes parameter ($\sigma, r, T$), and the option price on the $z$-axis. To aid interpretation of the resulting surfaces, a color gradient legend is included alongside each plot. Additionally, three red guide lines are drawn at $S_0 / E = \{0.8, \, 1.0, \, 1.2\}$ to highlight the differences in option price behavior under these scenarios.

The following abbreviations will be used from here onrwards:
- Out-the-money $\to$ OTM
- At-the-money $\to$ ATM
- In-the-money $\to$ ITM

In [ ]:
PLOTS_3D_DIR = 'plots_3d'

def surface_param_moneyness(
    strike,
    base_params,
    param_name,
    param_values,
    moneyness_values,
    base_dir=PLOTS_3D_DIR
):
    """
    Generates 3D surface plots for European and Binary call prices over a chosen Black–Scholes parameter and varying moneyness.
    """

    if param_name not in ('volatility', 'risk_free_rate', 'time_to_expiry'):
        raise ValueError("param_name must be 'volatility', 'risk_free_rate', or 'time_to_expiry'")

    sigma0 = base_params['volatility']
    r0     = base_params['rate']
    T0     = base_params['expiry']

    # Build grids
    Param, Moneyness = np.meshgrid(param_values, moneyness_values)
    prices = {'binary': np.zeros_like(Param),
              'european': np.zeros_like(Param)}

    for i in range(Param.shape[0]):
        for j in range(Param.shape[1]):
            val = Param[i, j]
            sigma, r, TT = sigma0, r0, T0
            if param_name == 'volatility':
                sigma = val
            elif param_name == 'risk_free_rate':
                r = val
            else:
                TT = val

            S = Moneyness[i, j] * strike
            opt = OptionPricing(
                terminal_prices=None,
                strike=strike,
                spot=S,
                T=TT,
                vol=sigma,
                rate=r
            )
            prices['binary'][i, j]   = opt.bs_binary()
            prices['european'][i, j] = opt.bs_european()

    out_dir = os.path.join(base_dir, param_name)
    os.makedirs(out_dir, exist_ok=True)

    paths = {}
    guide_moneyness = (0.8, 1.0, 1.2)

    for kind, cmap, title, zlabel in [
        ('binary',   'plasma',  'Binary Call Option Price',   'Binary Call Price'),
        ('european', 'viridis', 'European Call Option Price', 'European Call Price')
    ]:
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')

        surf = ax.plot_surface(
            Param,
            Moneyness,
            prices[kind],
            cmap=cmap,
            edgecolor='none',
            alpha=0.9
        )

        ax.set_xlabel(param_name.replace('_', ' ').capitalize())
        ax.set_ylabel('Moneyness (S/E)')
        ax.set_zlabel(zlabel, labelpad=10)  
        ax.set_title(title)

        fig.colorbar(surf, ax=ax, pad=0.1, shrink=0.6, aspect=15)

        # Red guide lines at fixed moneyness
        for money in guide_moneyness:
            line_vals = []
            for p in param_values:
                sigma, r, TT = sigma0, r0, T0
                if param_name == 'volatility':
                    sigma = p
                elif param_name == 'risk_free_rate':
                    r = p
                else:
                    TT = p
                S_line = money * strike
                op_line = OptionPricing(
                    terminal_prices=None,
                    strike=strike,
                    spot=S_line,
                    T=TT,
                    vol=sigma,
                    rate=r
                )
                line_vals.append(
                    op_line.bs_binary() if kind == 'binary' else op_line.bs_european()
                )
            ax.plot(
                param_values,
                [money] * len(param_values),
                line_vals,
                color='red',
                linewidth=2
            )

        plt.tight_layout()
        fig.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.9)

        # Save figure
        filename = f"{param_name}_{kind}.png"
        path = os.path.join(out_dir, filename)
        plt.savefig(path, bbox_inches='tight')
        plt.close(fig)
        paths[kind] = path

    return paths['european'], paths['binary']

In [ ]:
base_3d_params = {
    'volatility': 0.2,
    'rate': 0.05,
    'expiry': 1.0
}

moneyness_vals = np.linspace(0.6, 1.4, 50)

## 7.1 Volatility 

In [ ]:
surface_param_moneyness(
    strike=100,
    base_params=base_3d_params,
    param_name='volatility',
    param_values=np.linspace(0.01, 1.0, 50),
    moneyness_values=moneyness_vals,
)

display_plots('volatility', PLOTS_3D_DIR)

**European option**: The option price derivative with respect to $\sigma$ affects both terms in the Black-Scholes formula, with the most significant impact on the term proportional to $N(d_1)$, which is related to the option’s sensitivity to changes in volatility. The effect of volatility on the option price can be summarized as follows:

- If the option is deep ITM, the value of $d_1$ is large and positive, implying that the option price is less sensitive to volatility changes. As a result, increases in $\sigma$ have a smaller effect on the option price.

- If the option is deep OTM, the value of $d_1$ is negative, and the option price is highly sensitive to volatility. In this case, an increase in $\sigma$ increases the probability of the option finishing ITM, thus increasing the option value.

- ATM options experience the most significant change in price with respect to volatility, as the option is most sensitive to small changes in volatility when $S_0 \approx E$.

In general, increasing volatility leads to higher European call option prices, but the effect is more pronounced when the option is closer to being ATM.


**Binary option**: Different behaviors can be observed as volatility increases, and the reason lies in the behavior of the factor $d_2$, which is defined as
$$
d_2 = \frac{\ln\left(\frac{S_0}{E}\right) + \left(r - \frac{1}{2}\sigma^2\right)T}{\sigma \sqrt{T}}
$$
Since the binary option price is essentially proportional to $N(d_2)$, and $N(x)$ is a strictly increasing function, the effect of volatility can be analyzed by examining the partial derivative
$$
\frac{\partial d_2}{\partial \sigma} = -\frac{\sqrt{T}}{\sigma^2 T} \left[ \sigma^2 T + \ln\left(\frac{S_0}{E}\right) + \left(r - \frac{1}{2}\sigma^2\right)T \right]
$$
The stationary point $\frac{\partial d_2}{\partial \sigma} = 0$ can occur only if $\frac{S_0}{E} < e^{-rT}$, which implies that above this threshold the volatility-price curve is necessarily monotonic, and specifically decreasing. In fact, the derivative is negative when $\frac{S_0}{E} > e^{-rT - \frac{1}{2}\sigma^2 T}$, which always lies below the stationary threshold.

By plugging in the values $r = 0.05$ and $T = 1$, one finds that the threshold is approximately $\frac{S_0}{E} \approx 0.95$.


## 7.2 Risk-free rate

In [ ]:
surface_param_moneyness(
    strike=100,
    base_params=base_3d_params,
    param_name='risk_free_rate',
    param_values=np.linspace(0.04, 0.06, 50),
    moneyness_values=moneyness_vals,
)

display_plots('risk_free_rate', PLOTS_3D_DIR)

**European option**: The option price derivative with respect to $r$ only affects the term proportional to $N(d_2)$, and it is easy to show that this derivative is always positive. As a result, the option value increases for any initial condition of the spot price.

**Binary option**: Depending on the initial spot price relative to the strike, different scenarios can occur. In this case, the derivative computation is less straightforward, but some qualitative observations can be made:

- If the option is deep OTM, the price is highly sensitive to the probability of expiring ITM. A small increase in $N(d_2)$ due to a higher $r$ has a large relative effect. Therefore, the option price increases with higher risk-free rates.

- If the option is deep ITM, $N(d_2) \approx 1$, so it cannot increase much further. However, the discount factor $e^{-rT}$ continues to decrease with increasing $r$, and this effect dominates, causing the option value to decrease.


## 7.3 Time to expiry

In [ ]:
surface_param_moneyness(
    strike=100,
    base_params=base_3d_params,
    param_name='time_to_expiry',
    param_values=np.linspace(0.02, 2, 50),
    moneyness_values=moneyness_vals,
)

display_plots('time_to_expiry', PLOTS_3D_DIR)

**European option**: As $T$ increases, the option generally becomes more valuable due to the convexity of the payoff and the increased chance for the option to end up ITM. This behavior is confirmed mathematically by computing the partial derivative of the price with respect to $T$. For European calls, this derivative is negative for short maturities (i.e., time decay), but the total option value increases when comparing very short $T$ to longer horizons.

**Binary option**: This derivative expression is rather complex, but qualitatively it can be said that:
- If the option is deep ITM, then $N(d_2) \approx 1$, and increasing $T$ decreases the price due to the discount factor $e^{-rT}$.
- If the option is OTM, increasing $T$ gives more time for the option to potentially become in the money, increasing $N(d_2)$ and thus the overall price, unless the discounting dominates.

There is no simple closed-form threshold for when the binary option price switches from increasing to decreasing with $T$, but it depends critically on the relative position of $S_0$ and $E$, and the balance between the increase in $N(d_2)$ and the decay from $e^{-rT}$.


---

# 8. Conclusions

This project examined the use of Monte Carlo methods for pricing European and binary call options under the risk-neutral measure, by simulating the dynamics of the underlying stock using various numerical schemes. It began with an overview of Monte Carlo techniques and their relevance in option pricing, followed by the implementation of three methods for generating terminal stock prices: the Euler–Maruyama discretization, the Milstein scheme, and the exact closed-form solution. A convergence study was also conducted for the Euler and Milstein schemes, confirming that both share the same weak convergence order of 1, while the Milstein method exhibits superior strong convergence (order 1 compared to 0.5 for Euler).

The characteristics and payoff structures of European and binary call options were then reviewed, along with their analytical pricing formulas under the Black–Scholes framework. The results produced by the three numerical schemes were validated by examining the relative difference between Monte Carlo and Black–Scholes values, as well as the outcomes of $t$-tests. With all parameters held constant, all methods yielded highly accurate and consistent results.  

In the sensitivity analysis, the key parameters — volatility $\sigma$, risk‐free rate $r$, time to expiry $T$, spot price $S_0$, strike price $K$, as well as the number of simulations, and the number of time steps — were varied individually to examine their influence on option prices. To assess the methods performance, the option value obtained using them was compared to the Black-Scholes price using both the relative price distance and the $t$-test.
Across all parameter values, the Euler, Milstein, and closed‐form methods produced nearly identical price estimates for both European and binary options, likely due to the large number of simulations and fine time discretization. Although the Milstein scheme is theoretically more accurate in terms of path approximation, this improvement did not manifest in the pricing results, as pathwise errors were not analyzed. Similarly, the closed‐form method, which provides an exact solution to the stochastic differential equation, is expected to offer the least bias; however, no clear advantage was observed under the tested conditions.  

Asymptotic behaviors were then examined, with particular attention to the limiting behavior of option prices rather than the performance of the Monte Carlo methods; in these cases, only Black–Scholes values were considered. A final analysis of moneyness was conducted to investigate how pricing curves evolve with variations in the ratio $S_0/K$, observing that the option price could either increase or decrease in the different scenarios.

---

# 9. References

- Jäckel, P. (2002) - Monte Carlo Methods in Finance
- https://hautahi.com/sde_simulation